# Adding Trading Indicators for more data

In [1]:
import pandas as pd
import numpy as np
import ta

df = pd.read_csv('data/btc_usdt_futures_data.csv')

df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df = df.astype(float)
df

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2021-01-01 00:00:00,28948.19,29055.00,28706.00,29015.00,8037.588,1.609463e+12,2.321646e+08,65942.0,3983.842,1.151312e+08,0.0
2021-01-01 01:00:00,29015.00,29546.42,28975.46,29448.40,19543.401,1.609466e+12,5.736394e+08,127126.0,10386.352,3.048663e+08,0.0
2021-01-01 02:00:00,29443.65,29500.00,29159.16,29237.06,10049.720,1.609470e+12,2.946641e+08,69148.0,4513.355,1.323593e+08,0.0
2021-01-01 03:00:00,29236.58,29379.41,29174.95,29302.11,5579.452,1.609474e+12,1.633982e+08,47082.0,2969.050,8.697145e+07,0.0
2021-01-01 04:00:00,29302.11,29422.32,29086.00,29237.07,8351.683,1.609477e+12,2.443911e+08,59332.0,3661.258,1.071895e+08,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 20:00:00,65514.90,65973.50,65471.60,65756.90,8209.158,1.718399e+12,5.398276e+08,92264.0,4096.390,2.694044e+08,0.0
2024-06-14 21:00:00,65756.90,66648.80,65716.70,66179.90,16976.849,1.718402e+12,1.124104e+09,186711.0,9116.435,6.036847e+08,0.0
2024-06-14 22:00:00,66179.90,66241.80,65882.50,66011.00,6687.434,1.718406e+12,4.414046e+08,82119.0,3201.738,2.113226e+08,0.0


## BTC Historical Volatility

In [2]:
# Calculate log returns
df['log_return'] = np.log(df['close'] / df['close'].shift(1))

# Calculate rolling standard deviation of log returns to get realized volatility
window_size = 24  # 24 hours window for 1 day realized volatility
df['realized_volatility'] = df['log_return'].rolling(window=window_size).std()

# Annualize the volatility
df['hv'] = df['realized_volatility'] * np.sqrt(365 * 24)

## Garman-Klass Volatility

In [3]:
# Calculate Garman-Klass Volatility
def garman_klass_volatility(high, low, open, close, window=10):
    log_hl = np.log(high / low)
    log_co = np.log(close / open)
    rs = 0.5 * log_hl ** 2 - (2 * np.log(2) - 1) * log_co ** 2
    return np.sqrt(rs.rolling(window=window).sum())

In [4]:
df['gkv'] = garman_klass_volatility(df['high'], df['low'], df['open'], df['close'])

## Relative Strength Index (RSI)

In [5]:
# Calculate RSI
df['rsi'] = ta.momentum.RSIIndicator(df['close']).rsi()

## Bollinger Bands

In [6]:
# Calculate Bollinger Bands
bollinger = ta.volatility.BollingerBands(df['close'])
df['bb_high'] = bollinger.bollinger_hband()
df['bb_low'] = bollinger.bollinger_lband()

## Average True Range (ATR)

In [7]:
# Calculate ATR
df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close']).average_true_range()

## Moving Average Convergence Divergence (MACD)

In [8]:
# Calculate MACD
macd = ta.trend.MACD(df['close'])
df['MACD'] = macd.macd()
df['MACD_signal'] = macd.macd_signal()
df['MACD_diff'] = macd.macd_diff()

## Dollar Volume

In [9]:
# Calculate Dollar Volume
df['Dollar_volume'] = df['close'] * df['volume']

## Moving Average (MA)

In [10]:
# Calculate Moving Averages
df['ma_20'] = ta.trend.sma_indicator(df['close'], window=24)

## Exponential Moving Average (EMA)

In [11]:
df['ema_20'] = ta.trend.ema_indicator(df['close'], window=24)

## Stochastic Oscillator

In [12]:
# Calculate Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
df['stoch_k'] = stoch.stoch()
df['stoch_d'] = stoch.stoch_signal()

## Average Directional Index (ADX)

In [13]:
# Calculate Average Directional Index (ADX)
adx = ta.trend.ADXIndicator(df['high'], df['low'], df['close'])
df['adx'] = adx.adx()
df['adx_neg'] = adx.adx_neg()
df['adx_pos'] = adx.adx_pos()

## Commodity Channel Index (CCI)

In [14]:
# Calculate Commodity Channel Index (CCI)
df['cci'] = ta.trend.cci(df['high'], df['low'], df['close'], window=24)

## On-Balance Volume (OBV)

In [15]:
# Calculate On-Balance Volume (OBV)
df['obv'] = ta.volume.on_balance_volume(df['close'], df['volume'])

## Chaikin Money Flow (CMF)

In [16]:
# Calculate Chaikin Money Flow (CMF)
df['cmf'] = ta.volume.chaikin_money_flow(df['high'], df['low'], df['close'], df['volume'])

## Williams %R

In [17]:
# Calculate Williams %R
df['williams'] = ta.momentum.WilliamsRIndicator(df['high'], df['low'], df['close']).williams_r()

##  Parabolic SAR

In [18]:
# Calculate Parabolic SAR
df['parabolic_sar'] = ta.trend.PSARIndicator(df['high'], df['low'], df['close']).psar()

## Volume Weighted Average Price (VWAP)

In [19]:
# Calculate Volume Weighted Average Price (VWAP)
df['vwap'] = ta.volume.volume_weighted_average_price(df['high'], df['low'], df['close'], df['volume'])

## Fibonacci Retracement

In [20]:
# Calculate Fibonacci retracement levels
def fibonacci_retracement(df, period=24):
    high = df['high'].rolling(window=period).max()
    low = df['low'].rolling(window=period).min()
    diff = high - low
    levels = {
        'fibonacci_0.236': high - diff * 0.236,
        'fibonacci_0.382': high - diff * 0.382,
        'fibonacci_0.5': high - diff * 0.5,
        'fibonacci_0.618': high - diff * 0.618,
        'fibonacci_0.764': high - diff * 0.764
    }
    return pd.DataFrame(levels)

In [21]:
# Add Fibonacci levels to DataFrame
fibonacci_levels = fibonacci_retracement(df)
df = pd.concat([df, fibonacci_levels], axis=1)

## Save to CSV

In [22]:
# Select columns to exclude the first 11 columns
selected_columns_df = df.iloc[:, 13:]

selected_columns_df

,hv,gkv,rsi,bb_high,bb_low,atr,MACD,MACD_signal,MACD_diff,Dollar_volume,...,obv,cmf,williams,parabolic_sar,vwap,fibonacci_0.236,fibonacci_0.382,fibonacci_0.5,fibonacci_0.618,fibonacci_0.764
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,2.332106e+08,...,8037.588,NaN,NaN,29015.000000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,5.755219e+08,...,27580.989,NaN,NaN,29448.400000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,2.938243e+08,...,17531.269,NaN,NaN,29055.000000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1.634897e+08,...,23110.721,NaN,NaN,29159.160000,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,2.441787e+08,...,14759.038,NaN,NaN,29379.410000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-14 20:00:00,0.420216,0.017898,35.736333,67828.123239,65279.756761,484.414092,-420.134999,-301.497868,-118.637131,5.398088e+08,...,-3322249.418,-0.218923,-69.556199,66886.018501,66156.725082,66820.3724,66482.5138,66209.45,65936.3862,65598.5276
2024-06-14 21:00:00,0.433192,0.019810,44.112984,67800.934859,65243.985141,516.391657,-384.193572,-318.037009,-66.156563,1.123526e+09,...,-3305272.569,-0.235058,-51.276954,66776.001391,66145.569734,66820.3724,66482.5138,66209.45,65936.3862,65598.5276
2024-06-14 22:00:00,0.434724,0.019831,41.771664,67756.498585,65194.361415,505.170824,-365.129564,-327.455520,-37.674044,4.414442e+08,...,-3311960.003,-0.255385,-58.575688,66672.585308,66114.660224,66820.3724,66482.5138,66209.45,65936.3862,65598.5276


In [23]:
# Save the DataFrame to a CSV file
selected_columns_df.to_csv("data/indicators.csv")
print("Data saved to data/indicators.csv")

Data saved to data/indicators.csv
